# 1. Introdução

A análise abaixo foi realizada utilizando dados do campeonato mundial de Formula 1 (1950 - 2023) que foram disponibilizados no sites www.kaggle.com.

Perguntas que tentaremos responder:
- As cinco nacionalidades que mais contribuíram com pilotos na história da Fórmula 1 
- As 5 equipes com mais vitórias na Fórmula 1
- Os 5 pilotos com mais vitória na Fórmula 1
- Brasileiros com mais participações na Formula 1
.<br>
.<br>
.<br>
- ...

Fonte do dataset: https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020

In [ ]:
#!pip install kaggle

In [ ]:
#!pip install selenium

## 2. Carregando bibliotecas

In [ ]:
import pandas as pd
import kaggle
import numpy as np

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from urllib.parse import unquote

## 3. Carregando datasets

In [ ]:
drivers = pd.read_csv('drivers.csv')
results = pd.read_csv('results.csv')
constructors = pd.read_csv('constructors.csv')
races = pd.read_csv("races.csv")
drivers.head()

In [ ]:
drivers.query('driverId == 13')

In [ ]:
drivers.shape

In [ ]:
drivers.isnull().sum()

In [ ]:
drivers['driverId'].duplicated().sum()

#### Criando uma nova coluna com o nome inteiro do piloto

In [ ]:
drivers['fullName'] = drivers['forename'] + " " + drivers['surname']

##### Total de pilotos brasileiro que já disputaram a formula 1

In [ ]:
drivers.query('nationality == "Brazilian"').shape[0]

### Top 5 nacionalidades que mais contribuíram com pilotos na história da Fórmula 1

In [ ]:
top_5_drivers_nationality = drivers.groupby('nationality').count().sort_values('driverId', ascending=False)['driverId'].head()
top_5_drivers_nationality.plot(kind='bar', title="Top 5 nationality")
top_5_drivers_nationality

### Results CSV

In [ ]:
results.head(10)

#### Adicionando as colunas fullName e nationality no dataframe results

In [ ]:
results = results.merge(drivers[['fullName', 'driverId', 'nationality']], on='driverId')

#### Adicionando a coluna year no dataframe results

In [ ]:
results = results.merge(races[["year", "raceId"]], on="raceId")

In [ ]:
results.head()

### Top 5 brasileiros com maior número vitórias

In [ ]:
top_5_brazilian_results_p1 = results.query('nationality == "Brazilian" and position == "1"').groupby("fullName").count().sort_values("driverId", ascending=False)['resultId'].head()
top_5_brazilian_results_p1

In [ ]:
top_5_brazilian_results_p1.plot(kind='bar', title='Top 5 Brazilians winners', xlabel='Name')

### Brasileiros com maiores participações na Formula 1

In [ ]:
top_5_brazilian_parcitipations = results.query('nationality == "Brazilian"').groupby("fullName").count().sort_values("driverId", ascending=False)['resultId'].head()
top_5_brazilian_parcitipations

In [ ]:
top_5_brazilian_parcitipations.plot(kind='bar', title="Top 5 brazilian participations", xlabel="Name")

### Ayrton Senna vs Alain Prost

#### Os anos que Ayrton Senna e Alain Prost correram na Fórmula 1

In [ ]:
anos_na_formula1_ayrton_senna = results.query("fullName == 'Ayrton Senna'").sort_values("year", ascending=True)["year"].unique()
anos_na_formula1_alain_prost = results.query("fullName == 'Alain Prost'").sort_values("year", ascending=True)["year"].unique()
print("Ayrton Senna esteve em {} na Fórmula 1".format(anos_na_formula1_ayrton_senna))
print("-=" * 55)
print("Alain Prost esteve em {} na Fórmula 1".format(anos_na_formula1_alain_prost))

#### Total de corridas

In [ ]:
print("Total de corridas de Ayrton Senna: {}".format(results.query('fullName == "Ayrton Senna"').shape[0]))
print("Total de corridas de Alain Prost: {}".format(results.query('fullName == "Alain Prost"').shape[0]))

#### Total de corridas ganhas

In [ ]:
ayrton_senna_p1 = results.query('position == "1" and fullName == "Ayrton Senna"').count()['driverId']
alain_prost_p1 = results.query('position == "1" and fullName == "Alain Prost"').count()['driverId']
print("Total de corridas ganhas de Ayrton Senna: {}".format(ayrton_senna_p1))
print("Total de corridas ganhas de Alain Prost: {}".format(alain_prost_p1))


#### Total de corridas ganhas por corridas disputadas

In [ ]:
print("{}".format(ayrton_senna_p1 / results.query('fullName == "Ayrton Senna"').shape[0]))
print("{}".format(alain_prost_p1 / results.query('fullName == "Alain Prost"').shape[0]))

#### Total de pódios

In [ ]:
# Fazer de primeiro, segundo e terceiro colocado
ayrton_senna_podio =  results.query('(position == "1" or position == "2" or position == "3" ) and fullName == "Ayrton Senna"').count()['driverId']
alain_prost_podio = results.query('(position == "1" or position == "2" or position == "3" ) and fullName == "Alain Prost"').count()['driverId']
print("Pódios de Ayrton Senna: {}".format(ayrton_senna_podio))
print("Pódios de Alain Prost: {}".format(alain_prost_podio))
podios = { "Ayrton Senna": [ayrton_senna_podio],
            "Alain Prost": [alain_prost_podio]
         }

df_podios = pd.DataFrame(podios)
df_podios.plot(kind="bar")

#### Número de ocorrencias por posição de chegada

In [ ]:
corridas_podio_ayrton_senna = results.query('fullName == "Ayrton Senna"').groupby("position").count().sort_values("driverId", ascending=False)["driverId"].head()
corridas_podia_alain_prost = results.query('fullName == "Alain Prost"').groupby("position").count().sort_values("driverId", ascending=False)["driverId"].head()

In [ ]:
print("Ayrton Senna")
for item in range(corridas_podio_ayrton_senna[1:4].shape[0]):
    print("{}° colocado: {} corridas.".format(item+1, corridas_podio_ayrton_senna[1:4][item]))
    
print("-=" * 20)    
print("\n\n")

print("Alain Prost")
for item in range(corridas_podia_alain_prost[1:4].shape[0]):
    print("{}° colocado: {} corridas.".format(item+1, corridas_podia_alain_prost[1:4][item]))

In [ ]:
index = ["1° Colocado","2° Colocado","3° Colocado"]
ocorrencia_podio = pd.DataFrame({"Ayrton Senna": corridas_podio_ayrton_senna[1:4].values,
                   "Alain Prost": corridas_podia_alain_prost[1:4].values}, index=index)
ocorrencia_podio.plot(kind="bar", title="Pódio: Senna vs Prost", rot=0)

In [ ]:
constructors.head()

#### Adicionando o nome da equipe no dataframe results

In [ ]:
results = results.merge(constructors[['name', 'constructorId']], on='constructorId')
results.query('fullName == "Felipe Massa"').head()

### Top 5 equipes com maior número de vitórias

In [ ]:
top_5_contructors_winner = results.query('position == "1"').groupby('name').count().sort_values('resultId', ascending=False).head()['resultId']
top_5_contructors_winner

In [ ]:
### Duvidas
a = '\\N'
abc = results.query("position != @a")
abc = abc[['grid', 'position']].astype(int)

ooo = ["grid", "position"]
abc[ooo].corr()

In [ ]:
top_5_contructors_winner.plot(kind="bar", title="Top 5 constructor winners", xlabel="Constructors")

#### Excluindo colunas desnecessárias para o projeto e coluna "year" que foi adicionado ao dataframe para fazer algumas análises

In [ ]:
columns_drop_results = ["number", "grid", "positionText", "positionOrder", "points", "laps", "time", "milliseconds",
                       "fastestLap", "rank", "fastestLapTime", "fastestLapSpeed", "statusId","year", "name"]
results.drop(columns_drop_results, axis=1, inplace=True)

In [ ]:
results.head()

#### Substituindo o valor \\N para 0
- \\\N significa que o piloto não terminou a corrida

In [ ]:
results['position'].replace("\\N", 0, inplace=True)

In [ ]:
results['position'].unique()

### Top 5 pilotos com maiores número de vitórias

In [ ]:
top_5_p1 = results.query('position == "1"').groupby(['fullName']).count().sort_values('resultId', ascending=False)['resultId'].head()
#top_5_p1['resultId'].plot(kind='bar', title='Top 5 winners', xlabel='Name')
top_5_p1

In [ ]:
top_5_p1.plot(kind='bar', title='Top 5 winners', xlabel='Name')

# 4. Selenium

### Web Scraping para extrair as fotos dos pilotos de acordo com a URL disponibilizada no dataset "drivers.csv" 

- A tarefa consiste em acessar uma URL específica associada a cada piloto de corrida, verificar se há uma imagem disponível do piloto nessa página e, em seguida, armazenar essa URL da imagem em um dicionário chamado url_image. 
- Cada chave neste dicionário será o driverId do piloto e o valor correspondente será a URL da imagem, caso exista. Se não houver uma imagem disponível na página do piloto, será salva uma imagem padrão para indicar essa ausência

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)
url_image = {}
x = 0
y = 0
for i in range(1, drivers.shape[0]):
    driver.get(drivers['url'][x])
    elem = driver.find_elements(By.CLASS_NAME, "mw-file-element")
    contador_limite = len(elem) - 1
    if len(elem) != 0:
        while drivers['surname'][x].split()[0] not in unquote(elem[y].get_attribute("src")) :
            if contador_limite == y:
                break
            y += 1
#         url_image[drivers["driverId"][x]] = unquote(elem[y].get_attribute("src"))
#     else:
#         url_image[drivers["driverId"][x]] = "https://www.promoview.com.br/uploads/images/unnamed%2819%29.png"
        
    if len(elem) == 0 or drivers['surname'][x].split()[0] not in unquote(elem[y].get_attribute("src")):
        url_image[drivers["driverId"][x]] = "https://www.promoview.com.br/uploads/images/unnamed%2819%29.png"
    else:
        url_image[drivers["driverId"][x]] = unquote(elem[y].get_attribute("src"))
    y = 0
    x += 1


#### transformando o dicionário  em um dataframe

In [ ]:
url_image_dataframe = pd.DataFrame(list(url_image.items()), columns=["driverId", "urlImage"])

#### Junção do dataframe "drivers" com as URLs das fotos dos pilotos 

In [ ]:
drivers = drivers.merge(url_image_dataframe, on="driverId")
drivers

### Web Scraping para extrair todos os campeões da Fórmula 1

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)
driver.get("https://www.esportesmotor.com/resultados/formula-1/todos-os-campeoes-da-formula-1/")
elem = driver.find_elements(By.TAG_NAME, "table")

In [ ]:
html_campeoes_formula1 = pd.read_html(elem[0].get_attribute('outerHTML'))
df_campeoes_formula1 = pd.DataFrame(html_campeoes_formula1[0])
top_5_campeos_formula1 = df_campeoes_formula1.groupby("Piloto")[["Ano"]].count().sort_values("Ano", ascending=False).head()
top_5_campeos_formula1.rename(columns={"Ano": "Conquistas"}, inplace=True)
top_5_campeos_formula1

In [ ]:
top_5_campeos_formula1[:10].plot(kind="barh").invert_yaxis()

#### Transformando o dataframe de campeões para csv

In [ ]:
df_campeoes_formula1.to_csv("champions.csv", index=False)

In [ ]:
drivers.drop("url", axis=1, inplace=True)

In [ ]:
drivers.to_csv("drivers_with_image.csv", index=False)

In [ ]:
drivers.query("fullName == 'Alain Prost'")

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)
driver.get("https://www.englishclub.com/vocabulary/world-countries-nationality.php")
elem = driver.find_elements(By.CLASS_NAME, "ec-table")

In [ ]:
html_nationality = pd.read_html(elem[0].get_attribute('outerHTML'))

In [ ]:
nationality_to_country = pd.DataFrame(html_nationality[0])
nationality_to_country['id'] = nationality_to_country.index + 1

In [ ]:
nationality_to_country.drop(["person"], axis=1,inplace=True)
nationality_to_country.rename(columns={"adjective": "nationality"})

In [ ]:
##nationality_to_country.merge(drivers[["driverId", "nationality"]], left_on="adjective", right_on="nationality")